In [ ]:
import pandas as pd
import folium
import numpy as np
import math
from pathlib import Path
out_path = Path('/Path/To/Out/')
data_path = Path('/Path/To/Metadata/')
meta_path = data_path.joinpath('MetaDataFile.csv')
df_md = pd.read_csv(meta_path)
df_md.drop_duplicates(subset='Link ID', inplace=True)
df_md.reset_index(inplace=True,drop=True)

map_1 = folium.Map(location=[32, 35],
                       zoom_start=8,
                       tiles='Stamen Terrain')
for i,link in df_md.iterrows():
    if math.isnan(link['Rx Site Latitude']):
        print('No metadata for link ' + str(link['Link ID']))
        continue
    else:
        folium.PolyLine([(link['Rx Site Latitude'], 
                          link['Rx Site Longitude']),
                         (link['Tx Site Latitude'], 
                          link['Tx Site Longitude'])], 
                        color="purple", opacity=0.7, popup=str(link['Link ID'])).add_to(map_1)
        
map_1.save(str(out_path.joinpath('link_map.html')))